# FocusGAN ECE 285 Final Project

## imports

## Visualize CAT2000 Dataset

## Dataloader and Device Setup

## Generator

## U-Net Generator 

## Discriminator 

## Loss Functions and Optimizers

## Trainer

## Results